In [76]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from collections import defaultdict
from scipy.misc import imread
from skimage.filter import canny
from scipy.ndimage.filters import sobel

In [2]:
import math

In [3]:
def region_of_interest(img, vertices):
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [34]:
def draw_lines(img,lines, color=[255, 0, 0], thickness = 2):
    for line in lines:
        for x1,y1, x2, y2 in line:
            cv2.line(img,(x1,y1),(x2,y2), color, thickness)
            
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1],3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img 




In [57]:
# I borrow this section from arjunmitrareddy github. 
def windowed_average(values, window=3) :
    if len(values) < window:
        return [values[-1]]
    
    return np.convolve(values, np.ones((window,))/window, mode='valid')
    

#keep in global because we need to average over a set of images in the video case 
left_lane_params = []
right_lane_params = []
 
def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    left_lane_points, right_lane_points = [], []
    if lines is not None:
        for line in lines:
            for x1,y1,x2,y2 in line:
                slope = ((y2-y1)/(x2-x1))
                if slope < 0:
                    left_lane_points.append([x1, y1])
                    left_lane_points.append([x2, y2])
                else:
                    right_lane_points.append([x1, y1])
                    right_lane_points.append([x2, y2])

        if len(left_lane_points) > 0:
            left_fit = np.polyfit([x[0] for x in left_lane_points], [x[1] for x in left_lane_points], 1).tolist()

            if len(left_lane_params) == 0:
                left_lane_params.append(left_fit)

            elif abs(left_fit[0]) > 0.5 and abs(left_fit[0]) < 1.0:
                left_lane_params.append(left_fit)

        if len(right_lane_points) > 0:
            right_fit = np.polyfit([x[0] for x in right_lane_points], [x[1] for x in right_lane_points], 1).tolist()
            if len(right_lane_params) == 0:
                right_lane_params.append(right_fit)

            elif abs(right_fit[0]) > 0.5 and abs(right_fit[0]) < 1.0:
                right_lane_params.append(right_fit)

        l_slope = windowed_average([x[0] for x in left_lane_params])[-1]
        l_intercept = windowed_average([x[1] for x in left_lane_params])[-1]
        l_y1 = img.shape[0]
        l_x1 = int((l_y1 - l_intercept)/l_slope)
        l_y2 = int(img.shape[0] * 0.6)
        l_x2 = int((l_y2 - l_intercept)/l_slope)
           
        r_slope = windowed_average([x[0] for x in right_lane_params])[-1]
        r_intercept = windowed_average([x[1] for x in right_lane_params])[-1]
        r_y1 = int(img.shape[0] * 0.6)
        r_x1 = int((r_y1 - r_intercept)/r_slope)
        r_y2 = img.shape[0]
        r_x2 = int((r_y2 - r_intercept)/r_slope)
        
        cv2.line(img, (l_x1, l_y1), (l_x2, l_y2), color, thickness)
        cv2.line(img, (r_x1, r_y1), (r_x2, r_y2), color, thickness)
def hough_lines_extrapolation(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1],3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img 

In [53]:
def ROI(img):
    # this ROI has vertices value that works for both two videos. 
    y_shape = img.shape[0]
    x_shape = img.shape[1]
    vertices = np.array([[x_shape/2, y_shape*0.6], [x_shape/2, y_shape*0.6],
            [x_shape, y_shape*1], [0, y_shape*1]], dtype=np.int32)
    return region_of_interest(img, [vertices])

In [58]:
def pipeline_lane(img):
    #Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    # These values I borrow from arjunmitrareddy github 
    threshold = 10   # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 100 #minimum number of pixels making up a line
    max_line_gap = 100    # maximum gap in pixels between connectable line segments
    # Define our parameters for Canny and apply
    low_threshold = 50
    high_threshold = 150
    if len(img.shape)>2:
        img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        img_gray = img
    kernel_size = 5
    blur_gray = cv2.GaussianBlur(img_gray,(kernel_size, kernel_size),0)
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
    edge_roi = ROI(edges)
    lines_h = hough_lines(edge_roi, rho, theta, threshold, min_line_length, max_line_gap)
    return lines_h


In [62]:
def pipeline_lane_extrapolation(img):
    #Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 10   # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 100 #minimum number of pixels making up a line
    max_line_gap = 100    # maximum gap in pixels between connectable line segments
    # Define our parameters for Canny and apply
    low_threshold = 50
    high_threshold = 150
    if len(img.shape)>2:
        img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        img_gray = img
    kernel_size = 5
    blur_gray = cv2.GaussianBlur(img_gray,(kernel_size, kernel_size),0)
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
    edge_roi = ROI(edges)
    lines_h = hough_lines_extrapolation(edge_roi, rho, theta, threshold, min_line_length, max_line_gap)
    return lines_h


In [59]:
def process_image(image):
    result = pipeline_lane(image)
    lanes = cv2.addWeighted(image,0.8,result,1,0)
    return lanes

In [63]:
def process_image_extrapolation(image):
    result = pipeline_lane_extrapolation(image)
    lanes = cv2.addWeighted(image,0.8,result,1,0)
    return lanes

In [60]:
white_output = 'test_video_outputs/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_video_outputs/solidWhiteRight.mp4
[MoviePy] Writing video test_video_outputs/solidWhiteRight.mp4


100%|███████████████████████████████████████▊| 221/222 [00:08<00:00, 26.88it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_video_outputs/solidWhiteRight.mp4 

Wall time: 9.44 s


In [61]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [64]:
white_output = 'test_video_outputs/solidYellowLeft.mp4'
clip1 = VideoFileClip("test_videos/solidYellowLeft.mp4")
white_clip = clip1.fl_image(process_image_extrapolation) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_video_outputs/solidYellowLeft.mp4
[MoviePy] Writing video test_video_outputs/solidYellowLeft.mp4


100%|███████████████████████████████████████▉| 681/682 [00:26<00:00, 25.53it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_video_outputs/solidYellowLeft.mp4 

Wall time: 27.9 s


In [65]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [70]:
def colorSelect(img):
    # segment by color threshold 
    color_thresholds = np.array([210,110,70])
    white_colors = np.array([255,255,255])
    return cv2.inRange(img, color_thresholds, white_colors)

In [85]:
def pipeline_lane_challenge(img):
    #Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 10   # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 100 #minimum number of pixels making up a line
    max_line_gap = 100    # maximum gap in pixels between connectable line segments
    # Define our parameters for Canny and apply
    low_threshold = 50
    high_threshold = 150
    img = colorSelect(img)    
    if len(img.shape)>2:
        img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        img_gray = img
    kernel_size = 5
    blur_gray = cv2.GaussianBlur(img_gray,(kernel_size, kernel_size),0)
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
    edge_roi = ROI(edges)
    lines_h = hough_lines_extrapolation(edge_roi, rho, theta, threshold, min_line_length, max_line_gap)
    return lines_h

In [86]:
def process_image_challenge(image):
    result = pipeline_lane_challenge(image)
    lanes = cv2.addWeighted(image,0.8,result,1,0)
    return lanes

In [87]:
challenge_output = 'test_video_outputs/challenge.mp4'
clip2 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip2.fl_image(process_image_challenge)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_video_outputs/challenge.mp4
[MoviePy] Writing video test_video_outputs/challenge.mp4


100%|████████████████████████████████████████| 251/251 [00:24<00:00, 12.04it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_video_outputs/challenge.mp4 

Wall time: 27.8 s


In [88]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))